In [3]:
!pip install opencv-python


In [4]:
import numpy
import time
import calendar
from six.moves import cPickle as pickle
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

In [30]:
imageSize = 32  #The size of the original image - in pixels - assuming this is a square image
channels = 3    #The number of channels of the image. A RBG color image, has 3 channels
classes = 10    #The number of classes available for this dataset
trainingDataSize = 50000    #The number of images in the training set
testDataSize = 10000        #The number of images in the test set
trainigDataFiles = ('./data_batch_1', './data_batch_2', './data_batch_3', './data_batch_4','./data_batch_5') #An array of filenames containing the training data set
testDataFile = './test_batch' #The filename containing the test set
pcaFileName = 'pca' #The PCA filename
svmFileName = 'svm' #The SVM filename

In [31]:
winSize = imageSize
blockSize = 12
blockStride = 4
cellSize = 4
nbins = 18
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = True
nlevels = 64
signedGradient = True
hog = cv2.HOGDescriptor((winSize,winSize),(blockSize, blockSize),(blockStride,blockStride),(cellSize,cellSize),nbins,derivAperture, winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels,signedGradient)

In [32]:
##SVM parameters:
##===============
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_RBF)
svm.setC(0.1)
svm.setGamma(0.1)

In [33]:
pcaDim = 3000

In [48]:
def loadData(filename):
    '''
    Load the data from the given filename

    Parameters:
    -----------
    filename: string
        The name of the file containing the data to load

    Returns:
    --------
    theSet['data']:     array of images
    theSet['labels']:   array of labels
    '''
    f = open(filename, 'rb')
    theSet = pickle.load(f, encoding='latin1')
    f.close()

    return theSet['data'], theSet['labels']

In [49]:
def convertImages(origImages):
    '''
    Convert the images from CIFAR-10 format, to an array of 10000 images each is 32 X 32 X 3 size

    Parameters:
    -----------
    origImages: array
        array of images in the CIFAR-10 format

    Returns:
    --------
    images:     array of images each in 32 X 32 X 3 size
    '''
    images = np.reshape(origImages,(-1, channels, imageSize, imageSize))
    images = np.transpose(images, (0,2,3,1))

    return images

In [50]:
def loadTestData(filename):
    '''
    Load the test data

    Parameters:
    -----------
    filename: string
        The name of the file containing the test data

    Returns:
    --------
    testImages: array of images of the test data
    testLabels: array of labels of the test data
    '''

    origTestImages, testLabels = loadData(filename)
    testImages = convertImages(origTestImages)

    return testImages, testLabels

In [51]:
def loadTrainingData(filenames):
    '''
    Load all the training data from all files

    Parameters:
    -----------
    filenames: array of string
        An array The name of the file containing the data to load

    Returns:
    --------
    trainingImages: array of the training set images
    trainingLabels: array of the training set labels
    '''

    #Pre-allocate the arrays
    trainingImages = np.zeros(shape=[trainingDataSize, imageSize, imageSize, channels], dtype=numpy.uint8)
    trainingLabels = np.zeros(shape=[trainingDataSize], dtype=int)

    start=0
    for fileName in filenames:
        origImages, labels = loadData(fileName)
        images = convertImages(origImages)

        numOfImages = len(images);
        end = start + numOfImages;
        trainingImages[start:end, :] = images
        trainingLabels[start:end] = labels
        start = end

    return trainingImages, trainingLabels

In [52]:
def currentTime():
    '''
    Returns the current time in seconds since EPOC
    Used to measure how much time each phase took

    Returns:
    --------
    the current time in second since EPOC
    '''

    return calendar.timegm(time.gmtime())

In [53]:
def calcHOG(images):
    '''
    Calculate the HOG descriptors of the given images

    Parameters:
    -----------
    images: an array of images
        The images to which a HOG calculation should be applied

    Returns:
    --------
    hogDescriptors: an array of HOG vectors, 5832 components each
    '''

    hogDescriptors = []
    for image in images:
        hogDescriptors.append( hog.compute(image) )

    hogDescriptors = np.squeeze(hogDescriptors)
    return hogDescriptors

In [54]:
print("Loading the training set..."),
tik = currentTime()
trainingImages, trainingLabels = loadTrainingData(trainigDataFiles)
print("Took: " + str(currentTime()-tik) + " sec" )

Loading the training set...
Took: 1 sec


In [55]:
print("Creating HOG descriptors from the training set..."),
tik = currentTime()
trainHogDescriptors = calcHOG(trainingImages)
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the training set...
Took: 12 sec


In [56]:
print("Reducing the dimension of the HOG descriptors to " + str(pcaDim) + "..."),
tik = currentTime()
pca = PCA(pcaDim)
trainHogProjected = pca.fit_transform(trainHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing the dimension of the HOG descriptors to 3000...
Took: 200 sec


In [58]:
print("Save it as a PCA file...")
tik = currentTime()
pcaFile = open(pcaFileName, 'wb')
pickle.dump(pca, pcaFile)
pcaFile.close()
print("Took: " + str(currentTime() - tik) + " sec")

Save it as a PCA file...
Took: 1 sec


In [59]:
print("Training the SVM model using the reduced HOG descriptor..."),
tik = currentTime()
svm.train(np.asarray(trainHogProjected), cv2.ml.ROW_SAMPLE, np.asarray(trainingLabels))
svm.save(svmFileName)
print("Took: " + str(currentTime() - tik) + " sec")

Training the SVM model using the reduced HOG descriptor...
Took: 1990 sec


In [60]:

print("Loading the test set..."),
tik = currentTime()
testImages, testLabels = loadTestData(testDataFile)
print("Took: " + str(currentTime() - tik) + " sec")

Loading the test set...
Took: 0 sec


In [61]:
print("Creating HOG descriptors from the test set..."),
tik = currentTime()
testHogDescriptors = calcHOG(testImages);
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the test set...
Took: 1 sec


In [62]:
print("Reducing the dimension of the HOG descriptors to " + str(pcaDim) + "..."),
tik = currentTime()
testHogProjected = pca.transform(testHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing the dimension of the HOG descriptors to 3000...
Took: 2 sec


In [63]:
print("Classifying the test set..."),
tik = currentTime()
testResponse = svm.predict(np.asarray(testHogProjected))[1].ravel()
print("Took: " + str(currentTime() - tik) + " sec")

Classifying the test set...
Took: 233 sec


In [64]:
print ("Confusion matrix:")
print ("=================")
confusionMatrix = confusion_matrix(testLabels, testResponse)
print(confusionMatrix)

Confusion matrix:
[[779   8  55  22  23   6  14   2  68  23]
 [ 21 840   6  11  13   5  16   2  34  52]
 [ 92  10 582  74  98  55  43  16  23   7]
 [ 40  12 115 516  73 142  46  28  15  13]
 [ 30  13  86  89 645  34  40  40  19   4]
 [ 18   6  97 211  61 532  25  37   7   6]
 [ 21  17  42  62  44  30 766   5  10   3]
 [ 13   5  43  50  79  61   9 710  13  17]
 [ 71  30  14  12  13   9   5   6 819  21]
 [ 28  55  14  20  12   9   8   9  28 817]]


In [65]:
print ("======================================")
accuracy = (np.asarray(testLabels) == testResponse).mean()
print("Percentage Accuracy: %.2f %%" % (accuracy*100))
print ("======================================")

Percentage Accuracy: 70.06 %
